In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
import wrds
import talib as ta
import datetime, time

from scipy import stats
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [4]:
crsp = pd.read_csv('data/return.zip', 
                   parse_dates=['date'], infer_datetime_format=True)

E:\ProgramData\Anaconda3\envs\quant\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
crsp[['PERMNO', 'PERMCO']] = crsp[['PERMNO', 'PERMCO']].astype(int)
crsp_name = pd.read_csv('data/crsp_name.csv', 
                   parse_dates=['namedt','nameendt'], infer_datetime_format=True)
crsp_name = crsp_name.drop('Unnamed: 0', axis=1)
crsp_name[['permno', 'shrcd', 'exchcd']] = \
crsp_name[['permno', 'shrcd', 'exchcd']].astype(int)

crsp_name['namedt'] = pd.to_datetime(crsp_name['namedt'])
crsp_name['nameendt'] = pd.to_datetime(crsp_name['nameendt'])
# if nameendt is missing then set to today date
crsp_name['nameendt'] = crsp_name['nameendt'].fillna(pd.to_datetime('today'))
crsp_merge = pd.merge(crsp, crsp_name, 'inner', left_on='PERMNO', right_on='permno')
crsp_merge = crsp_merge[(crsp_merge.date <= crsp_merge.nameendt) &
                       (crsp_merge.date >= crsp_merge.namedt)]
crsp_merge.RET = crsp_merge.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
crsp_merge.RETX = crsp_merge.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)

crsp_merge.RET = crsp_merge.RET.fillna(0)
crsp_merge.RETX = crsp_merge.RETX.fillna(0)

crsp_merge = crsp_merge.drop(['permno'], axis=1)

crsp_merge.columns = [i.lower() for i in crsp_merge.columns]

In [91]:
# crsp[['PERMNO', 'PERMCO']] = crsp[['PERMNO', 'PERMCO']].astype(int)
# crsp.RET = crsp.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
# crsp.RETX = crsp.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)

# crsp.RET = crsp.RET.fillna(0)
# crsp.RETX = crsp.RETX.fillna(0)
# crsp.columns = [i.lower() for i in crsp.columns]

In [6]:
ccm = pd.read_csv('data/ccm_hist.csv')
ccm = ccm.drop('Unnamed: 0', axis=1)
ccm = ccm[((ccm.linktype == 'LU') | (ccm.linktype == 'LC')) & \
   ((ccm.linkprim == 'P') | (ccm.linkprim == 'C'))]
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm.columns = ['gvkey', 'linkprim', 'liid', 'linktype', 'permno', 'permco', 'linkdt',
       'linkenddt']

In [7]:
fiscal_end_month = pd.read_csv('data/fiscal_end_month_list_unique.csv')

In [8]:
# link ccm & fiscal end month table
ccm1 = pd.merge(ccm, fiscal_end_month, on=['gvkey'])

In [114]:
ccm.query('permno == 77418')

,gvkey,linkprim,liid,linktype,permno,permco,linkdt,linkenddt
50012,25056,C,01,LC,77418.0,11412.0,1992-03-19,2018-06-15


In [10]:
crsp2 = pd.merge(crsp_merge, ccm1, how='left', on=['permno'])

In [26]:
# crsp_merge = crsp_merge.drop_duplicates()

In [107]:
crsp2.head()

,permno_x,date,ticker,comnam,permco,cusip,prc,ret,shrout,retx,gvkey,linkprim,liid,linktype,permno_y,linkdt,linkenddt,fs_month
0,14403,1973-01-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-27.125,0.179348,21952.0,0.179348,15510.0,P,90,LC,14403.0,1982-01-29,1999-05-21,A-Feb
1,14403,1973-02-28,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-30.625,0.157272,21952.0,0.129032,15510.0,P,90,LC,14403.0,1982-01-29,1999-05-21,A-Feb
2,14403,1973-03-30,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-35.125,0.146939,21952.0,0.146939,15510.0,P,90,LC,14403.0,1982-01-29,1999-05-21,A-Feb
3,14403,1973-04-30,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-36.500,0.039146,21952.0,0.039146,15510.0,P,90,LC,14403.0,1982-01-29,1999-05-21,A-Feb
4,14403,1973-05-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-42.500,0.164384,21952.0,0.164384,15510.0,P,90,LC,14403.0,1982-01-29,1999-05-21,A-Feb


In [11]:
crsp2 = crsp2.drop_duplicates()

In [12]:
# crsp2 = crsp2[['permno', 'date', 'ticker', 'comnam','prc', 'ret',
#        'shrout', 'retx', 'shrcd', 'exchcd', 'fs_month']]
crsp2 = crsp2[['permno', 'date', 'ticker', 'comnam','prc', 'ret',
       'shrout', 'retx', 'fs_month']]

In [28]:
crsp2.shape

(3404247, 9)

In [112]:
crsp2.query('ticker == "AOL"')

,permno,date,ticker,comnam,prc,ret,shrout,retx,fs_month
1598640,77418,1996-09-30,AOL,AMERICA ONLINE INC DEL,35.5000,0.173554,90765.0,0.173554,NaN
1598641,77418,1996-10-31,AOL,AMERICA ONLINE INC DEL,27.1250,-0.235915,93368.0,-0.235915,NaN
1598642,77418,1996-11-29,AOL,AMERICA ONLINE INC DEL,35.2500,0.299539,93368.0,0.299539,NaN
1598643,77418,1996-12-31,AOL,AMERICA ONLINE INC DEL,33.2500,-0.056738,93368.0,-0.056738,NaN
1598644,77418,1997-01-31,AOL,AMERICA ONLINE INC DEL,37.0000,0.112782,93987.0,0.112782,NaN
1598645,77418,1997-02-28,AOL,AMERICA ONLINE INC DEL,37.5000,0.013514,95858.0,0.013514,NaN
1598646,77418,1997-03-31,AOL,AMERICA ONLINE INC DEL,42.5000,0.133333,95858.0,0.133333,NaN
1598647,77418,1997-04-30,AOL,AMERICA ONLINE INC DEL,45.1250,0.061765,95858.0,0.061765,NaN
1598648,77418,1997-05-30,AOL,AMERICA ONLINE INC DEL,55.1250,0.221607,95858.0,0.221607,NaN
1598649,77418,1997-06-30,AOL,AMERICA ONLINE INC DEL,55.6250,0.009070,97853.0,0.009070,NaN


In [111]:
crsp2.query('permno == 77418').sort_values('date')

,permno,date,ticker,comnam,prc,ret,shrout,retx,fs_month
1598586,77418,1992-03-31,AMER,AMERICA ONLINE INC DEL,15.250,0.000000,5467.0,0.000000,NaN
1598587,77418,1992-04-30,AMER,AMERICA ONLINE INC DEL,11.750,-0.229508,5467.0,-0.229508,NaN
1598588,77418,1992-05-29,AMER,AMERICA ONLINE INC DEL,12.500,0.063830,5467.0,0.063830,NaN
1598589,77418,1992-06-30,AMER,AMERICA ONLINE INC DEL,13.500,0.080000,5481.0,0.080000,NaN
1598590,77418,1992-07-31,AMER,AMERICA ONLINE INC DEL,14.125,0.046296,5481.0,0.046296,NaN
1598591,77418,1992-08-31,AMER,AMERICA ONLINE INC DEL,13.500,-0.044248,5481.0,-0.044248,NaN
1598592,77418,1992-09-30,AMER,AMERICA ONLINE INC DEL,15.000,0.111111,5572.0,0.111111,NaN
1598593,77418,1992-10-30,AMER,AMERICA ONLINE INC DEL,16.750,0.116667,5572.0,0.116667,NaN
1598594,77418,1992-11-30,AMER,AMERICA ONLINE INC DEL,19.750,0.179104,5572.0,0.179104,NaN
1598595,77418,1992-12-31,AMER,AMERICA ONLINE INC DEL,29.250,0.481013,5737.0,0.481013,NaN


In [30]:
# crsp.query('ticker == "AOL"')#.set_index('date')['2001']
# crsp.query('ticker == "AOL"').permco.value_counts()
crsp3 = crsp2[crsp2.fs_month.notnull() == True]

In [31]:
crsp3.shape

(2813139, 9)

In [32]:
crsp3 = crsp3.assign(mv = crsp3.prc.abs() * crsp3.shrout)
crsp3 = crsp3.assign(ret_p1=crsp3.ret+1, retx_p1=crsp3.retx+1)

In [56]:
crsp3.head()

,permno,ticker,comnam,prc,ret,shrout,retx,fs_month,mv,ret_p1,retx_p1
date,,,,,,,,,,,
1973-01-31,14403,AAGIY,ANGLO AMERICAN GOLD INVT LTD,-27.125,0.179348,21952.0,0.179348,A-Feb,595448.0,1.179348,1.179348
1973-02-28,14403,AAGIY,ANGLO AMERICAN GOLD INVT LTD,-30.625,0.157272,21952.0,0.129032,A-Feb,672280.0,1.157272,1.129032
1973-03-30,14403,AAGIY,ANGLO AMERICAN GOLD INVT LTD,-35.125,0.146939,21952.0,0.146939,A-Feb,771064.0,1.146939,1.146939
1973-04-30,14403,AAGIY,ANGLO AMERICAN GOLD INVT LTD,-36.500,0.039146,21952.0,0.039146,A-Feb,801248.0,1.039146,1.039146
1973-05-31,14403,AAGIY,ANGLO AMERICAN GOLD INVT LTD,-42.500,0.164384,21952.0,0.164384,A-Feb,932960.0,1.164384,1.164384


In [104]:
# def calc_cumprod(series):
#     res = (1+series).cumprod()
#     if len(res) == 1:
#         return res
#     else:
#         return res.iloc[-1]

def fiscal_resample(df):
#     df = df.set_index('date')
    fsm = df.fs_month.iloc[0]
#     df['ret_p1'] = (1+df['ret_p1']).cumprod()
#     df['retx_p1'] = (1+df['retx_p1']).cumprod()
    res = df.resample(fsm).apply({'ret_p1':'cumprod',
                                 'retx_p1':'cumprod',})
    res = pd.concat([res, df.mv], axis=1)
    res = res.resample(fsm).last()
#                                  'mv':'last'})
    return res

In [95]:
test = crsp3.reset_index().set_index('permno').loc[crsp3.\
            permno.unique().tolist()[:5]].groupby('permno').apply(fiscal_resample)
test

ret_p1   retx_p1            mv
permno date                                        
12160  1977-07-31  1.069444  1.069444  6.857812e+03
       1978-07-31  1.233765  1.233765  8.460938e+03
       1979-07-31  1.452632  1.452632  1.206638e+04
       1980-07-31  1.492753  1.492753  1.794775e+04
       1981-07-31  0.922330  0.922330  1.696938e+04
       1982-07-31  1.426316  1.426316  2.432225e+04
       1983-07-31  1.022140  1.022140  2.486075e+04
14403  1973-02-28  1.364826  1.331522  6.722800e+05
       1974-02-28  1.956214  1.877551  1.262240e+06
       1975-02-28  1.022470  0.956522  1.207360e+06
       1976-02-29  0.617167  0.570455  6.887440e+05
       1977-02-28  0.711205  0.637450  4.390400e+05
       1978-02-28  1.232446  1.118749  4.911760e+05
       1979-02-28  1.197396  1.134079  5.570320e+05
       1980-02-29  3.970482  3.546798  1.975680e+06
       1981-02-28  1.059550  0.927778  1.832992e+06
       1982-02-28  0.884941  0.770959  1.146519e+05
       1983-02-28  1.898060  1.685439  1.932385e+06
       1984-02-29  1.280573  1.184333  2.288585e+06
       1985-02-28  0.650412  0.612840  4.789575e+04
       1986-02-28  1.104106  1.035716  4.960631e+04
       1987-02-28  1.095080  0.998085  4.951131e+04
       1988-02-29  1.093917  1.024953  5.074669e+04
       1989-02-28  0.907270  0.842697  1.069172e+05
       1990-02-28  1.481515  1.386668  1.019948e+05
       1991-02-28  0.659307  0.631410  6.440053e+04
       1992-02-29  1.047672  1.000001  1.180338e+05
       1993-02-28  0.693894  0.639594  7.549369e+04
       1994-02-28  2.179326  2.079362  1.569789e+05
       1995-02-28  1.118529  1.068701  1.038362e+05
...                     ...       ...           ...
15580  2004-12-31  1.033333  1.033333  6.576030e+04
       2005-12-31  0.870968  0.870968  5.773410e+04
       2006-12-31  1.096297  1.096297  6.358376e+04
       2007-12-31  0.472973  0.472973  3.054380e+04
       2008-12-31  0.530000  0.530000  1.657034e+04
63845  1973-12-31  0.622521  0.609195  9.774525e+04
       1974-12-31  1.358097  1.320753  1.405425e+05
       1975-12-31  1.225056  1.190478  1.858750e+05
       1976-12-31  1.266395  1.227999  2.466361e+05
       1977-12-31  1.373944  1.328988  3.613860e+05
       1978-12-31  1.136189  1.093138  4.440766e+05
       1979-12-31  1.246922  1.188342  5.958525e+05
       1980-12-31  1.086408  1.030190  6.308348e+05
       1981-12-31  0.795372  0.750916  4.929225e+05
       1982-12-31  1.067456  0.990245  6.351362e+05
       1983-12-31  0.912406  0.871921  6.057161e+05
       1984-12-31  1.064950  1.016950  6.165000e+05
       1985-12-31  1.531071  1.477778  1.312444e+06
       1986-12-31  0.820435  0.796994  1.098743e+06
       1987-12-31  0.698734  0.669811  7.580848e+05
       1988-12-31  1.383947  1.323944  9.564500e+05
       1989-12-31  1.351740  1.303191  1.234096e+06
       1990-12-31  0.787604  0.755102  9.392450e+05
       1991-12-31  0.927051  0.886486  8.359285e+05
       1992-12-31  1.350194  1.292684  1.085069e+06
       1993-12-31  0.768132  0.735849  8.030880e+05
       1994-12-31  0.966109  0.948719  8.121130e+05
       1995-12-31  1.031511  1.027027  8.443980e+05
       1996-12-31  0.919631  0.914473  7.844465e+05
       1997-12-31  1.007194  1.007194  7.900900e+05

[125 rows x 3 columns]

In [105]:
tic = time.perf_counter()
crsp3 = crsp3.reset_index().set_index('date')
result = crsp3.groupby('permno').apply(fiscal_resample)
toc = time.perf_counter()
print(toc-tic)

157.00527250000005


In [106]:
result.head()

ret_p1   retx_p1           mv
permno date                                       
10000  1986-10-31  0.178571  0.178571   3002.34375
       1987-10-31  0.280000  0.280000    851.59375
10002  1986-12-31  1.064515  1.064515  14540.62500
       1987-12-31  0.808082  0.808082  11750.00000
       1988-12-31  0.837500  0.837500   9840.62500

In [101]:
result.head()

ret_p1   retx_p1           mv
permno date                                       
10000  1986-10-31  0.178571  0.178571   3002.34375
       1987-10-31  0.280000  0.280000    851.59375
10002  1986-12-31  1.064515  1.064515  14540.62500
       1987-12-31  0.808082  0.808082  11750.00000
       1988-12-31  0.837500  0.837500   9840.62500

In [103]:
result.shape

(211960, 3)